In [ ]:
# import necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import xgboost
import shap
import scipy.stats as stats
from scipy.stats import shapiro
from statsmodels.stats import stattools as stools

In [ ]:
# import data sets

historical = pd.read_csv('historical-draft-stats.csv')
current = pd.read_csv('draft-predict.csv')

In [ ]:
# establish features and output. slice data set to drop rows where any feature is empty

features = ['Pick', 'eFG%', 'FT%', 'PTS', 'SOS', '3PAr', 'FTr']
output = ['NBA-PPG']

historical = historical.dropna(subset=features)

train, test = train_test_split(historical, test_size = 0.25, random_state = 36)

xtrain = train[features]
ytrain = train[output]

xtest = test[features]
ytest = test[output]

In [ ]:
# function to train and evaluate models

def scores(model):
    
    model.fit(xtrain, ytrain.values.ravel())
    y_pred = model.predict(xtest)
    
    print("Mean squared error: %.3f" % mean_squared_error(ytest, y_pred))
    print('R2 score: %.3f' % r2_score(ytest, y_pred))

    cv_score = cross_val_score(model, xtest, ytest.values.ravel(), cv = 3, scoring = 'r2')
    print("R2 cross validation score: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))
    
    y_results = []
    
    for i in y_pred:
        y_results.append(i)
        
    return(y_results)

In [ ]:
svr = SVR(kernel='rbf', gamma=1e-4, C=100, epsilon = .01)

y_svr = scores(svr)

In [ ]:
rf = RandomForestRegressor(random_state = 0, n_estimators = 200, criterion = 'mse')

y_rf = scores(rf)

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors = 25, weights = 'uniform')

y_knn = scores(knn)

In [ ]:
xgb = xgboost.XGBRegressor(objective = "reg:squarederror", random_state = 0, n_estimators = 50)

y_xgb = scores(xgb)

# Standardized residuals

In [ ]:
# function to convert model y_pred values into standardized residuals

def residuals(x):
    
    resid = [i for i in (ytest['NBA-PPG'] - x)]
    ssr = [i ** 2 for i in resid]
    
    ssr_sum = 0
    for i in ssr:
        ssr_sum += i
        
    stand_resid = []
    for i in resid:
        stand_resid.append(i / ((ssr_sum / (ytest.shape[0] - 2)) ** (1/2)))
    
    resid_list = []
    
    for i in stand_resid:
        resid_list.append(i)
        
    return resid_list

In [ ]:
svr_resid = residuals(y_svr)
rf_resid = residuals(y_rf)
knn_resid = residuals(y_knn)
xgb_resid = residuals(y_xgb)

In [ ]:
# function to find outliers in standardized residuals (points more than 2 stdev away from mean)

def outliers(x):
    
    np_list = np.array(x)
    stdev = np.std(np_list)
    mean = np.mean(np_list)

    outliers = 0
    for i in x:
        if i < mean - 2 * stdev:
            outliers += 1
        elif i > mean + 2 * stdev:
            outliers += 1

    outlier_percent = 1 - outliers / ytest.shape[0]
    outlier_string = "{:.3%}".format(outlier_percent)
    
    return outlier_string

In [ ]:
# plot standardized residuals

plt.style.use('fivethirtyeight')

resid_fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharey = 'row')

x = np.arange(ytest.shape[0])
ax1.xaxis.set_visible(False)
ax2.xaxis.set_visible(False)
ax3.xaxis.set_visible(False)
ax4.xaxis.set_visible(False)

ax1.scatter(x, svr_resid)
ax1.axhline(y = np.mean(svr_resid), label = "Average", color = 'purple')
ax1.axhline(y = np.mean(svr_resid) - 2 * np.std(svr_resid), color = 'orange')
ax1.axhline(y = np.mean(svr_resid) + 2 * np.std(svr_resid), label = "2 stdev from mean", color = 'orange')
ax1.set_title("SVM: %s" % outliers(svr_resid), size = 18, x = .485, ha = 'center')

ax2.scatter(x, rf_resid)
ax2.axhline(y = np.mean(rf_resid), color = 'purple')
ax2.axhline(y = np.mean(rf_resid) - 2 * np.std(rf_resid), color = 'orange')
ax2.axhline(y = np.mean(rf_resid) + 2 * np.std(rf_resid), color = 'orange')
ax2.set_title("RF: %s" % outliers(rf_resid), size = 18, x = .485, ha = 'center')

ax3.scatter(x, knn_resid)
ax3.axhline(y = np.mean(knn_resid), color = 'purple')
ax3.axhline(y = np.mean(knn_resid) - 2 * np.std(knn_resid), color = 'orange')
ax3.axhline(y = np.mean(knn_resid) + 2 * np.std(knn_resid), color = 'orange')
ax3.set_title("KNN: %s" % outliers(knn_resid), size = 18, x = .485, ha = 'center')

ax4.scatter(x, xgb_resid)
ax4.axhline(y = np.mean(xgb_resid), color = 'purple')
ax4.axhline(y = np.mean(xgb_resid) - 2 * np.std(xgb_resid), color = 'orange')
ax4.axhline(y = np.mean(xgb_resid) + 2 * np.std(xgb_resid), color = 'orange')
ax4.set_title("XGB: %s" % outliers(xgb_resid), size = 18, x = .485, ha = 'center')

resid_fig.legend(loc = (.22, .855), ncol=2, prop={'size': 12, "family": "Rockwell"})

resid_fig.suptitle("Standardized Residuals", weight = 'bold', size = 18, y = 1.12)

ax1.yaxis.set_ticks([-2.5, 0, 2.5])
ax3.yaxis.set_ticks([-2.5, 0, 2.5])

resid_fig.text(x = 0, y = 0,
    s = '________________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

resid_fig.text(x = 0, y = -.06,
    s = 'https://dribbleanalytics.blog                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

resid_fig.savefig('resid-fig.png', dpi = 400, bbox_inches = 'tight')

# Q-Q plot

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('fivethirtyeight')

qqplot = plt.figure()

ax1 = qqplot.add_subplot(221)
stats.probplot(svr_resid, dist="norm", plot=plt)
ax1.set_xlabel("")
ax1.set_xticklabels([])
ax1.set_ylabel("")
ax1.set_title("SVM")

ax2 = qqplot.add_subplot(222)
stats.probplot(rf_resid, dist="norm", plot=plt)
ax2.set_xlabel("")
ax2.set_xticklabels([])
ax2.set_ylabel("")
ax2.set_title("RF")
ax2.set_xticklabels([])
ax2.set_yticklabels([])

ax3 = qqplot.add_subplot(223)
stats.probplot(knn_resid, dist="norm", plot=plt)
ax3.set_xlabel("")
ax3.set_ylabel("")
ax3.set_title("KNN")

ax4 = qqplot.add_subplot(224)
stats.probplot(xgb_resid, dist = "norm", plot = plt)
ax4.set_xlabel("")
ax4.set_ylabel("")
ax4.set_title("XGB")
ax4.set_yticklabels([])

qqplot.text(0.5, -0.02, 'Theoretical Quantiles', ha='center', va='center', size = 18)
qqplot.text(0.01, 0.5, 'Ordered Values', ha='center', va='center', rotation='vertical', size = 18)

qqplot.text(x = 0, y = -0.05,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left')

qqplot.text(x = 0, y = -.1,
    s = 'https://dribbleanalytics.blog                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

qqplot.savefig('qqplot.png', dpi = 400, bbox_inches = 'tight')

# Shapiro-Wilk test

In [ ]:
print(shapiro(svr_resid))
print(shapiro(rf_resid))
print(shapiro(knn_resid))
print(shapiro(xgb_resid))

# Predict

In [ ]:
# function to take models and predict current draft's PPG

curr_pred = current[features]

def predict(model):
    
    y_pred = model.predict(curr_pred)
    
    for i, j in zip(current['Player'], y_pred):
        print(i, j)
        
    return y_pred

In [ ]:
svr_pred = predict(svr)

In [ ]:
# function to make bar graph from predictions

def make_plot(pred_list, player_list, height, model_name, x_pos, file_name):
    
    plt.style.use('fivethirtyeight')
    
    fig, ax = plt.subplots()
    
    x = range(len(pred_list))
    
    ax.bar(x, pred_list, width = .7, edgecolor = 'white', color = 'skyblue', linewidth = 4, label = 'Predicted')

    labels = player_list

    rects = ax.patches
    for rect, label in zip(rects, labels):
        height = height
        ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
        ha='center', va='bottom', rotation = 'vertical', color = 'black')

    if(model_name == 'Average'):
        fig.suptitle("%s predicted rookie year PPG" % model_name, weight = 'bold', size = 18)
    else:
        fig.suptitle("%s predicted rookie year PPG" % model_name.upper(), weight = 'bold', size = 18)
    ax.xaxis.set_visible(False)
    ax.set_ylabel("PPG")

    fig.text(x = x_pos, y = 0.03,
        s = '______________________________________________________________',
        fontsize = 14, color = 'grey', horizontalalignment='left')

    fig.text(x = x_pos, y = -.03,
        s = 'https://dribbleanalytics.blog                     ',
        fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

    fig.savefig('%s.png' % file_name, dpi = 400, bbox_inches = 'tight')

In [ ]:
make_plot(svr_pred[0:9], current['Player'][0:9], .5, 'svm', -0.05, 'svr_1')

In [ ]:
make_plot(svr_pred[9:18], current['Player'][9:18], .2, 'svm', 0, 'svr_2')

In [ ]:
make_plot(svr_pred[18:], current['Player'][18:], .2, 'svm', 0, 'svr_3')

In [ ]:
rf_pred = predict(rf)

In [ ]:
make_plot(rf_pred[0:9], current['Player'][0:9], .5, 'rf', -0.05, 'rf_1')

In [ ]:
make_plot(rf_pred[9:18], current['Player'][9:18], .2, 'rf', 0, 'rf_2')

In [ ]:
make_plot(rf_pred[18:], current['Player'][18:], .2, 'rf', 0, 'rf_3')

In [ ]:
knn_pred = predict(knn)

In [ ]:
make_plot(knn_pred[0:9], current['Player'][0:9], .5, 'knn', -0.05, 'knn_1')

In [ ]:
make_plot(knn_pred[9:18], current['Player'][9:18], .2, 'knn', 0, 'knn_2')

In [ ]:
make_plot(knn_pred[18:], current['Player'][18:], .2, 'knn', 0, 'knn_3')

In [ ]:
xgb_pred = predict(xgb)

In [ ]:
make_plot(xgb_pred[0:9], current['Player'][0:9], .5, 'xgb', -0.05, 'xgb_1')

In [ ]:
make_plot(xgb_pred[9:18], current['Player'][9:18], .2, 'xgb', 0, 'xgb_2')

In [ ]:
make_plot(xgb_pred[18:], current['Player'][18:], .2, 'xgb', 0, 'xgb_3')

In [ ]:
avg_pred = [(g + h + i + j) / 4 for g, h, i, j in zip(svr_pred, rf_pred, knn_pred, xgb_pred)]

for i, j in zip(current['Player'], avg_pred):
    print(i, j)

In [ ]:
make_plot(avg_pred[0:9], current['Player'][0:9], .5, 'Average', -0.05, 'avg_1')

In [ ]:
make_plot(avg_pred[9:18], current['Player'][9:18], .2, 'Average', 0, 'avg_2')

In [ ]:
make_plot(avg_pred[18:], current['Player'][18:], .2, 'Average', 0, 'avg_3')

# SHAP values

In [ ]:
# prepare shap plots

shap.initjs()
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('fivethirtyeight')

In [ ]:
svr_k = shap.kmeans(xtrain, 5)
# use shap.kmeans to allow KernelExplainer to run on fewer data points - recommended for speed

explainer = shap.KernelExplainer(svr.predict, svr_k)
shap_values = explainer.shap_values(xtrain)

shap.summary_plot(shap_values, xtrain)

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(xtrain)

shap.summary_plot(shap_values, xtrain)

In [ ]:
knn_k = shap.kmeans(xtrain, 5)

explainer = shap.KernelExplainer(knn.predict, knn_k)
shap_values = explainer.shap_values(xtrain)

shap.summary_plot(shap_values, xtrain)

In [ ]:
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(xtrain)

shap.summary_plot(shap_values, xtrain)